Adapted from https://github.com/intersystems-community/iris-vector-search/blob/main/demo/sql_demo.ipynb

## Vector Search with IRIS SQL
This tutorial covers how to use IRIS as a vector database. 

For this tutorial, we will use a dataset of 2.2k online reviews of scotch (
dataset from https://www.kaggle.com/datasets/koki25ando/22000-scotch-whisky-reviews) . With our latest vector database functionality, we can leverage the latest embedding models to run semantic search on the online reviews of scotch whiskeys. In addition, we'll be able to apply filters on columns with structured data. For example, we will be able to search for whiskeys that are priced under $100, and are 'earthy, smooth, and easy to drink'. Let's find our perfect whiskey!

In [1]:
!pip install -q pandas

In [2]:
!pip install -q sqlalchemy-iris

In [3]:
!pip install -q langchain langchain-community

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.2 requires numpy<2.1.0,>=2.0.0; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
blis 1.0.1 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [4]:
!pip install -q spacy

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.3.1 requires numpy<2,>=1; python_version < "3.12", but you have numpy 2.0.2 which is incompatible.
langchain 0.3.1 requires numpy<2,>=1; python_version < "3.12", but you have numpy 2.0.2 which is incompatible.


In [5]:
!python -m spacy download en_core_web_sm > /dev/null

In [6]:
import time

start = time.time()

In [7]:
import os, pandas as pd
# from sentence_transformers import SentenceTransformer
from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings
from sqlalchemy import create_engine, text

In [8]:
username = '_system'
password = 'SYS'
hostname = 'sql-embeddings-iris-1'
port = '1972' 
namespace = 'IRISAPP'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"

In [9]:
engine = create_engine(CONNECTION_STRING)

In [10]:
# Load the CSV file
df = pd.read_csv('https://raw.githubusercontent.com/intersystems-community/iris-vector-search/refs/heads/main/data/scotch_review.csv')
df = df.head(100)

In [11]:
df.head()

,Unnamed: 0,name,category,review.point,price,currency,description
0,1,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,$,"Magnificently powerful and intense. Caramels, ..."
1,2,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,$,What impresses me most is how this whisky evol...
2,3,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,$,There have been some legendary Bowmores from t...
3,4,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,$,With a name inspired by a 1926 Buster Keaton m...
4,5,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,$,"Captivating, enticing, and wonderfully charmin..."


In [12]:
# Clean data
# Remove the specified columns
df.drop(['currency'], axis=1, inplace=True)

# Drop the first column
df.drop(columns=df.columns[0], inplace=True)

# Remove rows without a price
df.dropna(subset=['price'], inplace=True)

# Ensure values in 'price' are numbers
df = df[pd.to_numeric(df['price'], errors='coerce').notna()]

# Replace NaN values in other columns with an empty string
df.fillna('', inplace=True)

In [13]:
df.head()

,name,category,review.point,price,description
0,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,"Magnificently powerful and intense. Caramels, ..."
1,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,What impresses me most is how this whisky evol...
2,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,There have been some legendary Bowmores from t...
3,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,With a name inspired by a 1926 Buster Keaton m...
4,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,"Captivating, enticing, and wonderfully charmin..."


Now, InterSystems IRIS supports vectors as a datatype in tables! Here, we create a table with a few different columns. The last column, 'description_vector', will be used to store vectors that are generated by passing the 'description' of a review through an embedding model.

In [14]:
with engine.connect() as conn:
    with conn.begin():# Load 
        sql = f""" 
                DROP TABLE IF EXISTS scotch_reviews
                """
        result = conn.execute(text(sql))

In [15]:

with engine.connect() as conn:
    with conn.begin():# Load 
        sql = f"""
                CREATE TABLE scotch_reviews (
        name VARCHAR(255),
        category VARCHAR(255),
        review_point INT,
        price DOUBLE,
        description VARCHAR(2000),
        description_vector VECTOR(DOUBLE, 96)
        )
                """
        result = conn.execute(text(sql))

In [16]:
# # Load a pre-trained sentence transformer model. This model's output vectors are of size 384
# model = SentenceTransformer('all-MiniLM-L6-v2') 

from langchain_community.embeddings.spacy_embeddings import SpacyEmbeddings
model = SpacyEmbeddings(model_name='en_core_web_sm')

In [17]:

# # Generate embeddings for all descriptions at once. Batch processing makes it faster
# embeddings = model.encode(df['description'].tolist(), normalize_embeddings=True)

embeddings = model.embed_documents(df['description'].tolist())

# Add the embeddings to the DataFrame
df['description_vector'] = embeddings


In [18]:
df.head()

,name,category,review.point,price,description,description_vector
0,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,"Magnificently powerful and intense. Caramels, ...","[0.13861975073814392, -0.07249858230352402, -0..."
1,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,What impresses me most is how this whisky evol...,"[0.1379469633102417, -0.21900497376918793, 0.0..."
2,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,There have been some legendary Bowmores from t...,"[0.11233799904584885, -0.22451381385326385, -0..."
3,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,With a name inspired by a 1926 Buster Keaton m...,"[0.0643119141459465, -0.06992331892251968, 0.0..."
4,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,"Captivating, enticing, and wonderfully charmin...","[-0.021280774846673012, -0.17424021661281586, ..."


In [19]:
len(df['description_vector'][0])

96

In [20]:
with engine.connect() as conn:
    with conn.begin():
        for index, row in df.iterrows():
            sql = text("""
                INSERT INTO scotch_reviews 
                (name, category, review_point, price, description, description_vector) 
                VALUES (:name, :category, :review_point, :price, :description, TO_VECTOR(:description_vector))
            """)
            conn.execute(sql, {
                'name': row['name'], 
                'category': row['category'], 
                'review_point': row['review.point'], 
                'price': row['price'], 
                'description': row['description'], 
                'description_vector': str(row['description_vector'])
            })


Let's look for a scotch that costs less than $100, and has an earthy and creamy taste.

In [21]:
description_search = "earthy and creamy taste"
# search_vector = model.encode(description_search, normalize_embeddings=True).tolist() # Convert search phrase into a vector
search_vector = model.embed_documents(description_search)[0] # Convert search phrase into a vector

In [22]:
with engine.connect() as conn:
    with conn.begin():
        sql = text("""
            SELECT TOP 3 * FROM scotch_reviews 
            WHERE price < 100
            ORDER BY VECTOR_DOT_PRODUCT(description_vector, TO_VECTOR(:search_vector)) DESC
        """)

        results = conn.execute(sql, {'search_vector': str(search_vector)}).fetchall()


In [23]:
print(results)

[('Ardbeg An Oa', 'Single Malt Scotch', 93, 60.0, "Ardbeg's first standard release in nearly a decade, An Oa is matured in virgin oak, Pedro Ximénez, and bourbon barrels, with component whiskies marri ... (117 characters truncated) ... . A soft and sweet palate entry is followed by hot peat, black tea, peppery cloves, and aniseed. Black pepper lingers through the long, smoky finish.", '.14418117702007293701,-.19574566185474395751,-.034764584153890609741,.0064694741740822792053,-.069490298628807067871,.45286297798156738281,.187862455 ... (1902 characters truncated) ... 96980667114257,-.097458913922309875488,.12467429786920547486,.20600463449954986572,.12689094245433807373,-.22729438543319702148,.11949978023767471313'), ('Chivas, 18 year old, 40%', 'Blended Scotch Whisky', 95, 70.0, 'An essay in balance on both the aroma and palate. Silky layers of delicate sweetness (honeyed vanilla, caramel, light toffee) permeate through clean, ... (88 characters truncated) ...  with a hint of dark ch

In [24]:
results_df = pd.DataFrame(results, columns=df.columns).iloc[:, :-1] # Remove vector
pd.set_option('display.max_colwidth', None)  # Easier to read description
results_df.head()

,name,category,review.point,price,description
0,Ardbeg An Oa,Single Malt Scotch,93,60.0,"Ardbeg's first standard release in nearly a decade, An Oa is matured in virgin oak, Pedro Ximénez, and bourbon barrels, with component whiskies married in the distillery's French oak 'Gathering Vat.' The nose offers sweet peat, smoky lemon rind, ginger, and angelica. A soft and sweet palate entry is followed by hot peat, black tea, peppery cloves, and aniseed. Black pepper lingers through the long, smoky finish."
1,"Chivas, 18 year old, 40%",Blended Scotch Whisky,95,70.0,"An essay in balance on both the aroma and palate. Silky layers of delicate sweetness (honeyed vanilla, caramel, light toffee) permeate through clean, delicate fruit (citrus, peach, currant), and subtle, complex dried spice. Clean finish, with a hint of dark chocolate. A very versatile, very drinkable blend which suits most moods and occasions. Indeed, a benchmark blended scotch. \r\n"
2,"Ardbeg Corryvreckan, 57.1%",Single Malt Scotch,96,85.0,"Powerful, muscular, well-textured, and invigorating. Even within the realm of Ardbeg, this one stands out. The more aggressive notes of coal tar, damp kiln, anise, and smoked seaweed are supported by an array of fruit (black raspberry, black cherry, plum), dark chocolate, espresso, molasses, bacon fat, kalamata olive, and warming cinnamon on the finish. Quite stunning!"


## Elapsed time

In [25]:
done = time.time()
elapsed = done - start
print(f'elapsed time: {elapsed:.3f}')

elapsed time: 5.945
